# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../WeatherPy/Output/cities.csv')
cities_df


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Chernyshevskiy,63.01,112.47,8.78,89,100,11.23,RU,1603947116
1,1,Ushuaia,-54.80,-68.30,42.80,87,90,27.51,AR,1603947052
2,2,Paris,48.85,2.35,52.00,87,75,9.17,FR,1603946544
3,3,Andenes,69.31,16.12,39.20,100,75,19.46,NO,1603947123
4,4,Rawson,-43.30,-65.10,47.35,52,0,7.67,AR,1603947124
...,...,...,...,...,...,...,...,...,...,...
547,547,Pontianak,-0.03,109.33,90.55,60,18,2.28,ID,1603947871
548,548,Mogadishu,2.04,45.34,80.62,77,3,8.81,SO,1603948046
549,549,Shatsk,54.03,41.71,33.46,86,0,8.95,RU,1603948048
550,550,Okato,-39.20,173.88,60.01,94,97,1.99,NZ,1603947821


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[['Latitude', 'Longitude']]
weights = cities_df['Humidity']
fig = gmaps.figure(zoom_level = 2, center = (0,0))
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

In [5]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
clean_df = cities_df.dropna()

perfect_df = clean_df[(60 < clean_df['Max Temp']) & (clean_df['Max Temp'] < 80)
                     &(clean_df['Humidity'] <= 33)
                     &(clean_df['Cloudiness'] <= 33)
                     &(clean_df['Wind Speed'] < 10)]
perfect_df



,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Qeshm,26.96,56.27,77.00,27,0,2.24,IR,1603947130
25,25,Zhangye,38.93,100.45,64.22,21,0,5.41,CN,1603947159
49,49,Sakakah,29.97,40.21,66.20,17,0,3.36,SA,1603947196
64,64,Kaifeng,34.79,114.35,66.20,25,0,4.47,CN,1603947033
81,81,Busselton,-33.65,115.33,79.00,30,0,1.59,AU,1603947250
127,127,Adrar,20.50,-10.07,74.52,20,0,9.31,MR,1603947326
164,164,Puerto Peñasco,31.32,-113.53,68.92,26,0,8.86,MX,1603947398
234,234,Elliot,-31.33,27.85,76.03,32,16,9.93,ZA,1603947517
260,260,Pishin,30.58,67.00,61.57,25,0,1.86,PK,1603947562
371,371,Arlit,18.74,7.39,73.99,19,0,6.29,NE,1603947746


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = perfect_df[['City','Country','Latitude','Longitude']].copy()
hotel_df['Hotel Name'] = ''
hotel_df
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}


for i, row in hotel_df.iterrows():
    
    city = row['City']
    x = row['Latitude']
    y = row['Longitude']
    params['location'] = f'{x},{y}' 
#     print(params)
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
        
    response = requests.get(base_url, params=params).json()
    results = response['results']
   
    try:
        hotel_name = hotel_df.loc[i, 'Hotel Name'] = results[0]['name']
        print(f'{city} : {hotel_name}')
          
    except:
        print(f'No hotels found within 5 km of {city}')
        hotel_df.loc[i, 'Hotel Name'] = np.nan

Qeshm : Kimia IV Hotel
Zhangye : Zhangye Lvzhou Holiday Inn
Sakakah : Fakhamat Al Orjoana Apartment
Kaifeng : Pullman Kaifeng Jianye
Busselton : Observatory Guest House
No hotels found within 5 km of Adrar
Puerto Peñasco : Playa Bonita Resort
Elliot : Rose Garden Dinner Bed and Breakfast
Pishin : Iqra Printing Press
Arlit : Hôtel Telwa Bungalow
Tabas : شرکت معماری مهراز شایگان آراد
Bilma : Expeditions Ténére voyages


In [16]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
8,Qeshm,IR,26.96,56.27,Kimia IV Hotel
25,Zhangye,CN,38.93,100.45,Zhangye Lvzhou Holiday Inn
49,Sakakah,SA,29.97,40.21,Fakhamat Al Orjoana Apartment
64,Kaifeng,CN,34.79,114.35,Pullman Kaifeng Jianye
81,Busselton,AU,-33.65,115.33,Observatory Guest House
164,Puerto Peñasco,MX,31.32,-113.53,Playa Bonita Resort
234,Elliot,ZA,-31.33,27.85,Rose Garden Dinner Bed and Breakfast
260,Pishin,PK,30.58,67.00,Iqra Printing Press
371,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow
373,Tabas,IR,33.60,56.92,شرکت معماری مهراز شایگان آراد


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))